# Longitudinal Analysis in FreeSurfer with Single Subject


## The Experiment
Structural MR images were collected from a single subject in a baseline condition and a dehydrated condition. The subject drank 3 L of water before the baseline scan and abstained from hydration before bed for 10 hours prior to next morning's follow up scan. We predicted cortical thickness should decrease with dehydration.  This jupyter notebook takes you through the process for testing this hypothesis.


## Analysis Pipeline

### 1. Base template creation
```bash
ls ${SUBJECTS_DIR} # first confirm subjects dir points to right place
recon-all -base sub.01.base -tp sub.01.bl.01 -tp sub.01.dh.02 -all -parallel
```
### 2. Edit the base template

```bash
~/Surfer/gems/launchfv -s sub.01.base
```

### 3. Recreate pial surfaces after edits

```bash
recon-all -autorecon-pial -base sub.01.base -tp sub.01.bl.01 -tp sub.01.dh.02 -parallel
```

### 4. Make sure everything looks good!
```bash
~/Surfer/gems/launchfv -s sub.01.base
```

### 5. Generate the individual time points

```bash
recon-all -long sub.01.bl.01 sub.01.base -all -qcache -measure thickness -fwhm 7
recon-all -long sub.01.dh.02 sub.01.base -all -qcache -measure thickness -fwhm 7
```

### 6. Calculate the thickness difference on the surface

```bash
mris_thickness_diff -trg_type curv -out $(pwd)/lh.thickness_diff fsaverage/surf/lh.inflated sub.01.dh.02.long.sub.01.base.03/surf/lh.thickness.fwhm7.fsaverage.mgh fsaverage/surf/lh.inflated sub.01.bl.01.long.sub.01.base.03/surf/lh.thickness.fwhm7.fsaverage.mgh

mris_thickness_diff -trg_type curv -out $(pwd)/rh.thickness_diff fsaverage/surf/rh.inflated sub.01.dh.02.long.sub.01.base.03/surf/rh.thickness.fwhm7.fsaverage.mgh fsaverage/surf/rh.inflated sub.01.bl.01.long.sub.01.base.03/surf/rh.thickness.fwhm7.fsaverage.mgh

```
## Now you are ready to view the results! Does it look like what you would expect?
![left](fs/dehydrated-baseline_sub01000.png)
![right](fs/dehydrated-baseline_sub01002.png)
![anterior](fs/dehydrated-baseline_sub01003.png)
![posterior](fs/dehydrated-baseline_sub01001.png)

### Ok, does this even mean anything? I dont know how to interpret this
Ok well it seems that there is a lot of noise from scan to scan and that any differences in the cortical sheet could be due to the algorithm, the image quality or other natural variations in physiology that have nothing to do with our experiment.  This is usually why we do group studies and look at averages across many subjects in order to cancel out as much noise as possible.

Let's try looking at some of the volumetric structures below the cortical sheet as these may be more sensitive to chances in hydration (i.e. ventricle size).

In [25]:
# Make sure you are using python 2 or this command won't work
asegstatsdiff sub.01.bl.01.long.sub.01.base.03 sub.01.dh.02.long.sub.01.base.03 | sed 's/\\t/  /g'


Runtime error (func=(main), adr=16): Divide by zero
(standard_in) 1: parse error

----------------
Aseg Stats Diff:
subject1: sub.01.bl.01.long.sub.01.base.03, subject2: sub.01.dh.02.long.sub.01.base.03
% diff:    label:   (subject1:value, subject2:value)
-------    -----------------------------------------
-10.2800    WM-hypointensities   (s1: 301.3, s2: 270.3)
-5.9300    Right-Accumbens-area   (s1: 772.8, s2: 726.9)
-4.3200    Left-Accumbens-area   (s1: 428.1, s2: 409.6)
-3.6900    CC_Mid_Posterior   (s1: 582.5, s2: 561.0)
-2.6800    Right-Cerebellum-White-Matter   (s1: 14009.5, s2: 13632.9)
-2.5700    CC_Central   (s1: 523.8, s2: 510.3)
-2.3500    Left-Hippocampus   (s1: 4779.7, s2: 4667.1)
-2.2900    Right-Thalamus-Proper   (s1: 7820.8, s2: 7641.5)
-2.1800    Left-Cerebellum-White-Matter   (s1: 15865.3, s2: 15519.1)
-1.9400    Brain-Stem   (s1: 22902.1, s2: 22455.9)
-1.8200    Left-Caudate   (s1: 4331.0, s2: 4251.8)
-1.5500    Right-Hippocampus   (s1: 4637.3, s2: 4565.2)
-1.4600   

## It appears that over all CSF volume increased from day 1 to day 2. Any possible explanations for this? (Hint: aquaporins)